In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image

In [12]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2020-04-29 18:33:12--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip.1’

concrete_data_week4 100%[===================>] 249.37M  35.6MB/s    in 7.8s    

2020-04-29 18:33:21 (32.0 MB/s) - ‘concrete_data_week4.zip.1’ saved [261483817/261483817]



In [ ]:
!unzip concrete_data_week4.zip

In [ ]:
negative_images = os.listdir('/content/concrete_data_week4/train/negative')
positive_images = os.listdir('/content/concrete_data_week4/train/positive')

In [ ]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input

In [ ]:
num_classes = 2
image_resize = 224
batch_size_training = 100
batch_size_validation = 100

In [ ]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [19]:
train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [20]:
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')

Found 9501 images belonging to 2 classes.


In [ ]:
model = Sequential()

In [22]:
model.add(ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

94658560/94653016 [==============================] - 3s 0us/step


In [ ]:
model.add(Dense(num_classes, activation='softmax'))

In [25]:
model.layers

In [ ]:
model.layers[0].trainable = False

In [27]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 4098      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 6         
Total params: 23,591,816
Trainable params: 4,104
Non-trainable params: 23,587,712
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

In [31]:
fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Epoch 1/2
301/301 [==============================] - 276s 916ms/step - loss: 0.5091 - accuracy: 0.9683 - val_loss: 0.3966 - val_accuracy: 0.8953
Epoch 2/2
301/301 [==============================] - 264s 875ms/step - loss: 0.3324 - accuracy: 0.9946 - val_loss: 0.2648 - val_accuracy: 0.9401


In [ ]:
model.save('classifier_resnet_model.h5')

In [ ]:
from keras.applications.vgg16 import VGG16

In [ ]:
model_vgg = Sequential()

In [ ]:
model_vgg.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

In [ ]:
model_vgg.add(Dense(num_classes, activation='softmax'))

In [ ]:
model_vgg.layers[0].trainable = False

In [46]:
model_vgg.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
model_vgg.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [48]:
fit_history_vgg = model_vgg.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Epoch 1/2
301/301 [==============================] - 243s 808ms/step - loss: 0.0873 - accuracy: 0.9728 - val_loss: 4.7684e-07 - val_accuracy: 0.9952
Epoch 2/2
301/301 [==============================] - 241s 801ms/step - loss: 0.0189 - accuracy: 0.9958 - val_loss: 0.0012 - val_accuracy: 0.9969


In [ ]:
model_vgg.save('classifier_vgg16_model.h5')

In [ ]:
data_generator_test = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [60]:
test_generator = data_generator_test.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    shuffle=False)

Found 500 images belonging to 2 classes.


In [61]:
model_vgg.evaluate_generator(test_generator,steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False)

[0.0062302956357598305, 0.9980000257492065]

In [62]:
model.evaluate_generator(test_generator, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False)

[0.3369705080986023, 0.9539999961853027]

In [ ]:
result=model_vgg.predict_generator(test_generator, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False)

In [71]:
for i in range(1,500,100):
  if result[i][0]>result[i][1]:
    print("Negative")
  else:
    print("Positive")

Negative
Negative
Negative
Positive
Positive
